# Verify MPR Data Loading and Standardization Pipeline

This notebook demonstrates the process of loading a BioLogic `.mpr` file, standardizing it, and inspecting the results using `echemistpy`.

In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr

# Ensure src is in path if running from examples directory without installation
project_root = Path("..").resolve()
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from echemistpy.utils.external.echem.biologic_reader import BiologicMPTReader
from echemistpy.io.standardized import standardize_measurement

In [ ]:
# Define file path
file_path = Path("echem/Biologic_GPCL.mpr")
print(f"Loading file: {file_path.resolve()}")

## 1. Load Data Using BiologicMPTReader

For `.mpr` files, we use the specialized `BiologicMPTReader` which can parse BioLogic binary files and returns a `RawMeasurement`.

In [ ]:
# Load using BiologicMPTReader
reader = BiologicMPTReader()
raw_measurement = reader.read(file_path)

print(f"Technique: {raw_measurement.metadata.meta.get('technique')}")
print(f"Sample Name: {raw_measurement.metadata.meta.get('sample_name')}")
print(f"Instrument: {raw_measurement.metadata.meta.get('instrument')}")

# Display dataset structure
print(f"\nData variables: {list(raw_measurement.data.data.data_vars.keys())}")
print(f"Data shape: {raw_measurement.data.data.dims}")

raw_measurement.data.data

## 2. Standardize Data

Apply the standardization pipeline to normalize column names and units.

In [ ]:
# Standardize
standardized_measurement, info = standardize_measurement(raw_measurement)

print(f"Standardized Technique: {info.technique}")
print(f"Sample Name: {info.sample_name}")
print(f"\nStandardized variables: {list(standardized_measurement.data.data_vars.keys())}")

# Display standardized dataset structure
standardized_measurement.data

## 3. Visualize Data

Plot the electrochemical data to verify it loaded correctly.

In [ ]:
# Plot Potential vs Time
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Check for time column (could be 'time/s' or 'Time/s')
time_col = None
for col in standardized_measurement.data.data_vars:
    if 'time' in col.lower() and '/s' in col.lower():
        time_col = col
        break

if time_col is None:
    # Use original column name
    time_col = 'time/s'

# Plot voltage
if 'Ewe/V' in standardized_measurement.data:
    axes[0].plot(standardized_measurement.data[time_col], 
                 standardized_measurement.data['Ewe/V'], 'b-')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Potential (V)')
    axes[0].set_title('Working Electrode Potential vs Time')
    axes[0].grid(True)
else:
    print("Ewe/V column not found")

# Plot current
current_col = None
for col in standardized_measurement.data.data_vars:
    if 'current' in col.lower() or col.startswith('<I>'):
        current_col = col
        break

if current_col:
    axes[1].plot(standardized_measurement.data[time_col], 
                 standardized_measurement.data[current_col], 'r-')
    axes[1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Current (mA)')
    axes[1].set_title('Current vs Time')
    axes[1].grid(True)
else:
    print("Current column not found")

plt.tight_layout()
plt.show()

## 4. Inspect Metadata

Check what metadata was extracted from the MPR file.

In [ ]:
# Display metadata
print("=== Measurement Metadata ===")
for key, value in info.extras.items():
    if not key.startswith('mpr_'):
        print(f"{key}: {value}")

print("\n=== MPR-Specific Metadata ===")
for key, value in info.extras.items():
    if key.startswith('mpr_'):
        if isinstance(value, (list, dict)) and len(str(value)) > 100:
            print(f"{key}: <{type(value).__name__} with {len(value)} items>")
        else:
            print(f"{key}: {value}")

## 5. Save Standardized Data (Optional)

Save the standardized data to NetCDF format for future use. We need to clean None values from attributes first.

In [ ]:
# Clean None values from attributes before saving to NetCDF
# NetCDF doesn't accept None as attribute values
ds_to_save = standardized_measurement.data.copy(deep=True)

# Clean variable attributes
for var in ds_to_save.data_vars:
    attrs_to_remove = [k for k, v in ds_to_save[var].attrs.items() if v is None]
    for attr in attrs_to_remove:
        del ds_to_save[var].attrs[attr]

# Clean coordinate attributes
for coord in ds_to_save.coords:
    attrs_to_remove = [k for k, v in ds_to_save[coord].attrs.items() if v is None]
    for attr in attrs_to_remove:
        del ds_to_save[coord].attrs[attr]

# Now save
ds_to_save.to_netcdf("standardized_biologic_gpcl.nc")
print("Data saved to standardized_biologic_gpcl.nc")